# Assignment 1


| Student Name      | Student Number |
| ----------- | ----------- |
| Baorong Huang      | N10172912       |



## Question 1

In [1]:
import glob
import string
from stemming.porter2 import stem
from math import sqrt, log


### Task 1.1

In [2]:
class BowDoc:
    def __init__(self, docID:str="", terms:dict[str, int]={}, doc_len:int=0):
        """Initialize a BowDoc object.

        @docId: A docID variable which is simply assigned by the value of 'itemid' in <newsitem …>.
        @terms: key-value pair of (String term: int frequency)
        @doc_len: The total number of words in a document.
        """
        self.docID = docID
        self.terms = terms
        self.doc_len = doc_len
    
    def getDocID(self):
        """Getter for docID"""
        return self.docID

    def getTerms(self):
        """Getter for terms"""
        return self.terms

    def setDocLen(self, doc_len):
        """Setter for doc_len"""
        self.doc_len = doc_len

    def getDocLen(self):
        """Getter for doc_len"""
        return self.doc_len
    
    def addTerm(self, new_term:str):
        """add new term or increase term frequency when the term occur again."""
        if new_term in self.terms:
            self.terms[new_term] += 1
        else:
            self.terms[new_term] = 1


In [3]:
def parse_rcv_coll(input_path:str, stop_words:list[str]) -> dict[str, BowDoc]:
    """Parse a data collection in the given folder and 
    build up a collection of BowDoc objects for the given dataset

    @input_path: The folder that contains the dataset in XML format.
    @stop_words: A list of common English words.

    @return: A collection of BowDoc objects. A dictionary structure with docID as key and BowDoc object as value. 
    """
    # build up a collection of BowDoc objects for the given dataset
    # docID as key and BowDoc object as value
    BowDoc_collection:dict[str, BowDoc] = {} 
    for file_path in glob.glob(f"{input_path}/*.xml"):
        with open(file_path, "r") as file:
            word_count = 0
            docID = ""

            def process_term(term:str):
                nonlocal word_count
                word_count += 1
                # Stopping words removal and add to BowDoc
                if len(term) > 2 and (term not in stop_words):
                    BowDoc_collection[docID].addTerm(term)

            text_tag_start = False
            for line in file:
                # remove \n and spaces
                line = line.strip()

                # obtain docID from itemid in <newsitem>
                if not docID:
                    if line.startswith("<newsitem "):
                        for part in line.split():
                            if part.startswith("itemid="):
                                docID = part.split("=")[1].strip("\"")
                                BowDoc_collection[docID] = BowDoc(docID,{})

                 # look for the content of <text></text>
                if line.startswith("<text>"):
                    text_tag_start = True
                    continue
                elif line.startswith("</text>"):
                    text_tag_start = False
                    BowDoc_collection[docID].setDocLen(word_count)
                    break
                
                # tokenize <text></text>
                if text_tag_start:
                    process_line(line, process_term)
                   
    return BowDoc_collection

### Task 1.2

In [4]:
def process_line(line:str, on_term_process):
    """Process a line of string.
    
    @line: A line of words.
    @on_term_process: A function that will be call back when processing a term.
    """
    # remove \n and spaces
    line = line.strip()

    # exclude p tags
    line = line.replace("<p>","").replace("</p>", "")

    # discard punctuations and/or numbers
    line = line.translate(str.maketrans("", "", string.digits))
    line = line.translate(str.maketrans(string.punctuation, " "* len(string.punctuation)))

    for term in line.split():
        term = stem(term.lower())
        on_term_process(term)

In [5]:
def parse_query(query0:str, stop_words:list[str]) -> dict[str, int]:
    """Parse a query and return a term frequency dictionary. {term, num_occurrences}
    
    @query0: A simple sentence or title.
    @stop_words: A list of stop words.

    @return: A term frequency dictionary. {term:frequency}
    """
    d = BowDoc(terms={})
    def process_term(term:str):
        # Stopping words removal and add to BowDoc
        if len(term) > 2 and (term not in stop_words):
            d.addTerm(term)
        
    process_line(query0, process_term)
    return d.getTerms()
    

### Task 1.3

In [13]:
def task_1():
    stopwords_f = open('common-english-words.txt', 'r')
    stop_words = stopwords_f.read().split(',')

    docs = parse_rcv_coll("Rnews_t120", stop_words)
    output = ""
    for docID, doc in sorted(docs.items()):
        output += f"Document {docID} contains {len(doc.getTerms())} indexing terms and have total {doc.getDocLen()} words\n"
        # sorts index terms (by frequency)
        sorted_terms = sorted(doc.getTerms().items(),reverse=True, key=lambda item: item[1])
        # prints out a term:freq list
        for term, count in sorted_terms:
            output += f"{term} : {count}\n"
    print(output)
    # saves the output into a text file
    # file_name = "Baorong_Huang_Q1.txt"
    # file = open(file_name, "w")
    # file.write(output)

task_1()

Document 67460 contains 14 indexing terms and have total 30 words
amp : 2
cowen : 1
cut : 1
rate : 1
minnesota : 1
mine : 1
manufactur : 1
neutral : 1
buy : 1
analyst : 1
avail : 1
comment : 1
stock : 1
down : 1
Document 69629 contains 144 indexing terms and have total 357 words
mine : 12
quot : 6
peopl : 5
kill : 5
worker : 5
ltd : 4
clash : 3
between : 3
south : 3
fight : 3
polic : 3
ethnic : 3
pondo : 3
sotho : 3
gold : 3
attack : 3
violenc : 3
month : 3
two : 2
blast : 2
explos : 2
monday : 2
buffelsfontein : 2
morn : 2
africa : 2
presid : 2
inquiri : 2
industri : 2
labour : 2
system : 2
union : 2
apartheid : 2
pillar : 2
day : 1
african : 1
stick : 1
knive : 1
appar : 1
spark : 1
rivalri : 1
erupt : 1
northwest : 1
johannesburg : 1
earli : 1
sunday : 1
took : 1
use : 1
underground : 1
deton : 1
against : 1
shack : 1
seven : 1
three : 1
stab : 1
beaten : 1
death : 1
tri : 1
flee : 1
carnag : 1
total : 1
eight : 1
attribut : 1
reveng : 1
crowd : 1
launch : 1
one : 1
six : 1
reason :

## Question 2

### Task 2.1

In [7]:
def calc_df(coll:dict[str, BowDoc]):
    """calculate document-frequency (df) for a given BowDoc collection coll and return a {term:df, …} dictionary. 
    @coll: A BowDoc collection. {docID:BowDoc}

    @return: A {term:df, …} dictionary.
    """
    # {term: count}
    df = {} 
    output = ""
    
    for doc in coll.values():
        terms = doc.getTerms()
        for term in terms.keys():
            if term in df:
                df[term] += 1
            else:
                df[term] = 1
    
    # sort them in descending order
    
    df = { k:v for k,v in sorted(df.items(), key=lambda item: item[1], reverse=True) }

    output += f"There are {len(coll)} documents in this data set and contains {len(df)} terms\n"
    for term, count in df.items():
        output += f"{term} : {count}\n"
    # print(output)
    return df

### Task 2.2

In [8]:
def tfidf(doc:BowDoc, df:dict[str, int], ndocs:int) -> dict[str, float]:
    """calculate tf*idf value (weight)
    
    @doc: A BowDoc instance.
    @df: A {term: df} dictionary.
    @ndocs: The number of documents in a given BowDoc collection.

    @ returns a {term:tfidf_weight , …} dictionary for the given document doc.
    """
    tf_idf = {}
    denominator = 0
    terms = doc.getTerms()
    for term, count in terms.items():
        denominator += ((log(count) + 1) * log(ndocs / df[term]))**2
    
    denominator = sqrt(denominator)
    for term, tf in doc.getTerms().items():
        numerator = (log(tf) + 1) * log(ndocs/df[term])
        tf_idf[term] = numerator / denominator

    return tf_idf

### Task 2.3

In [9]:
def task_2():
    stopwords_f = open('common-english-words.txt', 'r')
    stop_words = stopwords_f.read().split(',')
    docs = parse_rcv_coll("Rnews_t120", stop_words)
    
    # Number of documents
    N = len(docs)
    df = calc_df(docs)

    for doc in docs.values():
        terms = doc.getTerms()
        tf_idf = tfidf(doc, df, N)

        print(f"Document {doc.getDocID()} contains {len(terms)} terms")
        for term, weight in sorted(tf_idf.items(), key=lambda item: item[1], reverse=True):
            print(f"{term} : {weight}")
        print()

    queries = ["USA: RESEARCH ALERT - Minnesota Mining cut.", 
               "SOUTH AFRICA: Death toll reaches 24 in S.African mine clashes.",
               "SOUTH AFRICA: Three killed in new clashes at S.Africa gold mine."]
    for query in queries:
        # {docID:relevance}
        Q = parse_query(query, stop_words)
        R = {docID:0 for docID in docs}
        for doc in docs.values():
            terms = doc.getTerms()
            tf_idf = tfidf(doc, df, N)
            for term in Q:
                if term in tf_idf:
                    R[doc.getDocID()] += tf_idf[term] * Q[term]            
        
        print(f"The Ranking Result for query: {query}\n")
        for k,v in sorted(R.items(), key=lambda item:item[1] ,reverse=True):
            print(f"{k} : {v}")
        
        
task_2()
    

Document 78091 contains 45 terms
newsroom : 0.35131343097356754
southwest : 0.26298067729993324
renew : 0.25331702769099573
critic : 0.25331702769099573
affect : 0.25331702769099573
take : 0.1959928065651436
toll : 0.1959928065651436
weekend : 0.1959928065651436
thursday : 0.1959928065651436
anoth : 0.1959928065651436
klerksdorp : 0.1959928065651436
johannesburg : 0.15992001892205407
overnight : 0.15532062440842392
xhosa : 0.15532062440842392
near : 0.15532062440842392
mile : 0.15532062440842392
wednesday : 0.15532062440842392
output : 0.15532062440842392
buffelsfontein : 0.129022464327993
injur : 0.1237728289938519
randgold : 0.1237728289938519
explor : 0.1237728289938519
african : 0.0979964032825718
miner : 0.0979964032825718
sinc : 0.0979964032825718
sotho : 0.0979964032825718
worker : 0.0979964032825718
hostel : 0.0979964032825718
amp : 0.0979964032825718
violenc : 0.0979964032825718
three : 0.09705834337703137
gold : 0.09705834337703137
kill : 0.07620274587429761
ethnic : 0.076202

## Question 3

### Task 3.1

In [10]:
def avg_doc_len(coll:dict[str, BowDoc]) -> float:
    """calculate  and  return  the  average document length of all documents in the collection coll.
    
    @coll: A collection of BowDoc objects. {docID:BowDoc}.

    @return: The average document length.
    """
    total_len = 0
    N = len(coll)
    for doc in coll.values():
        total_len += doc.getDocLen()
    return total_len / N


### Task 3.2

In [11]:
def bm25(coll:dict[str, BowDoc], q:str, df:dict[str, int]) -> dict[str, float]:
    """Calculate  documents' BM25 score for a given original query q
    
    @coll: A collection of documents. {docID: BowDoc}.
    @q: The original query.
    @df: document frequency. {term:df}.

    @return: A dictionary of {docID:bm25_score} for all documents in collection coll.
    """
    stopwords_f = open('common-english-words.txt', 'r')
    stop_words = stopwords_f.read().split(',')
    scores = {}
    k_1 = 1.2
    k_2 = 100
    b = 0.75
    dl = 0
    N = 2*len(coll)
    # the average document length of a doc in the collection
    avdl = avg_doc_len(coll)
    query_term_frequency = parse_query(q, stop_words)

    for doc in coll.values():
        score = 0
        dl = doc.getDocLen()
        # frequency of terms in the document
        f = doc.getTerms()
        K = k_1 * ( (1 - b) + b *  dl/avdl)
        for term in query_term_frequency:
            f_i = 0
            n_i = 0
            if term in f:
                f_i = f[term]
            if term in df:
                n_i = df[term]
            qf_i = query_term_frequency[term]
            first_term = 1/( (n_i + 0.5)/(N - n_i + 0.5) )
            second_term = ((k_1 + 1)*f_i) / (K + f_i)
            third_term = ((k_2 + 1)*qf_i)/(k_2 + qf_i)

            score += log(first_term, 2) * second_term * third_term
        scores[doc.getDocID()] = score
    
    return scores
    

### Task 3.3

In [12]:
def task_3():
    queries = ["Deaths mining accidents", "Mentioning deaths in mining accidents", "Statistics on number of mining deaths", "ethnic clashes and resultant deaths of mine workers near a mine"]
    stopwords_f = open('common-english-words.txt', 'r')
    stop_words = stopwords_f.read().split(',')
    docs = parse_rcv_coll("Rnews_t120", stop_words)
    
    avdl = avg_doc_len(docs)
    print(f"Average document length for this collection is: {avdl}")
    df = calc_df(docs)
    for q in queries:
        print(f"The query is: {q}")
        bm25_scores = bm25(docs, q, df)
        print("The following are the BM25 score for each document:")
        for docID, score in sorted(bm25_scores.items(), key=lambda item:item[1], reverse=True):
            doc_len = docs[docID].getDocLen()
            print(f"Document ID: {docID}, Doc Length: {doc_len} -- BM25 Score: {score}")
        
        print("\nThe following are possibly relevant documents retrieved -")
        for docID, score in sorted(bm25_scores.items(), key=lambda item:item[1], reverse=True)[:5+1]:
            print(f"{docID} {score}")
        print()
task_3()

Average document length for this collection is: 340.5
The query is: Deaths mining accidents
The following are the BM25 score for each document:
Document ID: 75135, Doc Length: 265 -- BM25 Score: 2.016725388884778
Document ID: 78091, Doc Length: 90 -- BM25 Score: 1.748675953066234
Document ID: 78092, Doc Length: 163 -- BM25 Score: 1.5537357475577047
Document ID: 71406, Doc Length: 357 -- BM25 Score: 1.1986310135782883
Document ID: 69629, Doc Length: 357 -- BM25 Score: 1.1986310135782883
Document ID: 69633, Doc Length: 414 -- BM25 Score: 1.1232065781332516
Document ID: 75338, Doc Length: 428 -- BM25 Score: 1.1061112071306798
Document ID: 67460, Doc Length: 30 -- BM25 Score: 0.0
Document ID: 71759, Doc Length: 1082 -- BM25 Score: 0.0
Document ID: 86836, Doc Length: 129 -- BM25 Score: 0.0
Document ID: 81047, Doc Length: 499 -- BM25 Score: 0.0
Document ID: 71948, Doc Length: 272 -- BM25 Score: 0.0

The following are possibly relevant documents retrieved -
75135 2.016725388884778
78091 1.748